In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
df_train=pd.read_csv("/content/drive/My Drive/Colab Notebooks/RNN LSTM Fake News/train.csv")

In [5]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df_train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [7]:
x_train=df_train.drop(["target"], axis=1)

In [8]:
y_train=df_train["target"]

In [9]:
x_train.shape, y_train.shape

((7613, 4), (7613,))

In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [12]:
voc_size=1000

In [13]:
message=x_train.copy()

In [14]:
message["text"][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [27]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from nltk.stem import WordNetLemmatizer
lematizer= WordNetLemmatizer() 
nltk.download('wordnet')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [28]:
corpus=[]
for i in range(0,len(message["text"])):
  review=re.sub('[^a-zA-Z]'," ",message["text"][i])
  review=review.lower()
  review=review.split()
  review=[lematizer.lemmatize(ps.stem(word)) for word in review if not word in stopwords.words('english')]
  review=" ".join(review)
  corpus.append(review)

In [29]:
corpus

['deed reason earthquak may allah forgiv u',
 'forest fire near la rong sask canada',
 'resid ask shelter place notifi offic evacu shelter place order expect',
 'peopl receiv wildfir evacu order california',
 'got sent photo rubi alaska smoke wildfir pour school',
 'rockyfir updat california hwi close direct due lake counti fire cafir wildfir',
 'flood disast heavi rain caus flash flood street manit colorado spring area',
 'top hill see fire wood',
 'emerg evacu happen build across street',
 'afraid tornado come area',
 'three peopl die heat wave far',
 'haha south tampa get flood hah wait second live south tampa gonna gonna fvck flood',
 'rain flood florida tampabay tampa day lost count',
 'flood bago myanmar arriv bago',
 'damag school bu multi car crash break',
 'man',
 'love fruit',
 'summer love',
 'car fast',
 'goooooooaaaaaal',
 'ridicul',
 'london cool',
 'love ski',
 'wonder day',
 'looooool',
 'way eat shit',
 'nyc last week',
 'love girlfriend',
 'cooool',
 'like pasta',
 'e

In [30]:
from tensorflow.keras.preprocessing.text import one_hot
one_hot_rep=[one_hot(word,voc_size) for word in corpus]

In [31]:
one_hot_rep

[[511, 956, 593, 662, 771, 281, 644],
 [67, 410, 423, 749, 56, 856, 407],
 [688, 621, 581, 78, 221, 372, 530, 581, 78, 820, 725],
 [46, 274, 83, 530, 820, 456],
 [806, 655, 601, 68, 943, 371, 83, 101, 227],
 [853, 847, 456, 138, 115, 993, 184, 859, 436, 410, 318, 83],
 [821, 212, 96, 21, 458, 812, 821, 808, 161, 233, 53, 543],
 [193, 941, 479, 410, 774],
 [935, 530, 168, 344, 322, 808],
 [104, 199, 580, 543],
 [699, 46, 255, 407, 329, 177],
 [478, 194, 805, 70, 821, 426, 82, 263, 1, 194, 805, 895, 895, 407, 821],
 [21, 821, 385, 349, 805, 23, 103, 786],
 [821, 290, 399, 875, 290],
 [591, 227, 637, 832, 592, 537, 360],
 [370],
 [723, 290],
 [471, 723],
 [592, 758],
 [942],
 [916],
 [976, 937],
 [723, 397],
 [321, 23],
 [946],
 [559, 574, 424],
 [544, 546, 637],
 [723, 332],
 [27],
 [619, 882],
 [39],
 [318, 10, 489, 21, 707, 18, 4, 938],
 [958, 551, 255, 96, 915, 652, 707, 18, 524, 71, 168],
 [948, 360, 860, 827, 589, 272, 21, 651, 707, 18, 22],
 [139, 272, 21],
 [436, 697, 168, 210, 54

In [32]:
sent_length=max([len(sen.split()) for sen in corpus])
sent_length

27

In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
emb_doc=pad_sequences(one_hot_rep,maxlen=sent_length,padding='pre')

In [34]:
emb_doc

array([[  0,   0,   0, ..., 771, 281, 644],
       [  0,   0,   0, ...,  56, 856, 407],
       [  0,   0,   0, ...,  78, 820, 725],
       ...,
       [  0,   0,   0, ...,  18, 416, 113],
       [  0,   0,   0, ...,   6, 601, 977],
       [  0,   0,   0, ...,  18, 129, 234]], dtype=int32)

In [35]:
emb_doc[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0, 511, 956, 593, 662, 771, 281,
       644], dtype=int32)

In [39]:
embeding_vect_features=50
model=Sequential()
model.add(Embedding(voc_size,embeding_vect_features,input_length=sent_length))
model.add(Dropout(0.2))
model.add(LSTM(500))
model.add(Dropout(0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss="binary_crossentropy",optimizer='Adam',metrics=['accuracy'])

In [40]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 27, 50)            50000     
_________________________________________________________________
dropout_4 (Dropout)          (None, 27, 50)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 500)               1102000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 501       
Total params: 1,152,501
Trainable params: 1,152,501
Non-trainable params: 0
_________________________________________________________________


In [41]:
import numpy  as np
x_train=np.array(emb_doc)
y_train=np.array(y_train)

In [42]:
x_train.shape, y_train.shape

((7613, 27), (7613,))

In [43]:
model.fit(x_train,y_train,batch_size=64,epochs=10)

Epoch 1/10
119/119 [==============================] - 9s 9ms/step - loss: 0.6491 - accuracy: 0.6234
Epoch 2/10
119/119 [==============================] - 1s 8ms/step - loss: 0.4914 - accuracy: 0.7682
Epoch 3/10
119/119 [==============================] - 1s 8ms/step - loss: 0.4390 - accuracy: 0.8086
Epoch 4/10
119/119 [==============================] - 1s 8ms/step - loss: 0.4323 - accuracy: 0.8069
Epoch 5/10
119/119 [==============================] - 1s 8ms/step - loss: 0.3941 - accuracy: 0.8294
Epoch 6/10
119/119 [==============================] - 1s 8ms/step - loss: 0.3886 - accuracy: 0.8313
Epoch 7/10
119/119 [==============================] - 1s 8ms/step - loss: 0.3701 - accuracy: 0.8416
Epoch 8/10
119/119 [==============================] - 1s 8ms/step - loss: 0.3607 - accuracy: 0.8448
Epoch 9/10
119/119 [==============================] - 1s 8ms/step - loss: 0.3421 - accuracy: 0.8510
Epoch 10/10
119/119 [==============================] - 1s 8ms/step - loss: 0.3259 - accuracy: 0.8651

In [44]:
df_test=pd.read_csv("/content/drive/My Drive/Colab Notebooks/RNN LSTM Fake News/test.csv")

In [45]:
df_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [46]:
test_message=df_test.copy()
test_message

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [48]:

test_corpus=[]
for i in range(0,len(test_message["text"])):
  review=re.sub('[^a-zA-Z]'," ",test_message["text"][i])
  review=review.lower()
  review=review.split()
  review=[lematizer.lemmatize(ps.stem(word)) for word in review if not word in stopwords.words('english')]
  review=" ".join(review)
  test_corpus.append(review)

In [49]:
test_corpus

['happen terribl car crash',
 'heard earthquak differ citi stay safe everyon',
 'forest fire spot pond gee flee across street cannot save',
 'apocalyps light spokan wildfir',
 'typhoon soudelor kill china taiwan',
 'shake earthquak',
 'probabl still show life arsen yesterday eh eh',
 'hey',
 'nice hat',
 'fuck',
 'like cold',
 'nooooooooo',
 'tell',
 '',
 'awesom',
 'birmingham wholesal market ablaz bbc news fire break birmingham wholesal market http co irwqcezweu',
 'sunkxssedharri wear short race ablaz',
 'previouslyondoyintv toke makinwa marriag crisi set nigerian twitter ablaz http co cmghxba xi',
 'check http co roi nsmejj http co tj zjin http co yduixefip http co lxtjc kl nsfw',
 'psa split person techi follow ablaz co burner follow ablaz',
 'bewar world ablaz sierra leon amp guap',
 'burn man ablaz turban diva http co hodwosamw via etsi',
 'dis song peopl take thing run smh eye open though set game ablaz cyhitheprync',
 'rape victim die set ablaz year old girl die burn injuri se

In [50]:
test_one_hot_rep=[one_hot(word,voc_size) for word in test_corpus]

In [51]:
test_emb_doc=pad_sequences(test_one_hot_rep,maxlen=sent_length,padding='pre')

In [52]:
test_emb_doc

array([[  0,   0,   0, ..., 178, 592, 537],
       [  0,   0,   0, ..., 644, 357, 472],
       [  0,   0,   0, ..., 808, 839, 164],
       ...,
       [  0,   0,   0, ..., 707,  18, 613],
       [  0,   0,   0, ...,  18, 303, 938],
       [  0,   0,   0, ..., 935, 198, 391]], dtype=int32)

In [53]:
x_test=np.array(test_emb_doc)

In [55]:
y_predict=model.predict_classes(test_emb_doc)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [56]:
y_predict

array([[0],
       [0],
       [1],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [57]:
output=pd.DataFrame(df_test["id"],columns=["id"])
output["target"]=y_predict

In [58]:
output

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,1


In [60]:
output.to_csv("/content/drive/My Drive/Colab Notebooks/RNN LSTM Fake News/sample_submission1.csv",index=False)